In [1]:
import pandas as pd
from datetime import datetime
import gspread
import os

In [2]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            #print("Jupyter notebook or qtconsole")
            return True
        elif shell == 'TerminalInteractiveShell':
            #print("Terminal running IPython")
            return False
        else:
            return False
    except NameError:
        return False

In [3]:
CREDS_FILE = 'service_account.json'
if is_notebook():
    import IPython
    HTML = IPython.display.HTML
    creds_file_path = CREDS_FILE
else:
    creds_file_path = os.path.dirname(os.path.realpath(__file__)) + '/' +  CREDS_FILE

In [4]:
gc = gspread.service_account(filename=creds_file_path)
sh = gc.open(str('transactions'))

worksheet = sh.worksheet('raw')
rdf = pd.DataFrame(worksheet.get_all_records())

In [5]:
report_sh = gc.open('Report')
report_id = 'all'

try:
    report_worksheet = report_sh.add_worksheet(title=report_id, rows=200, cols=14)
except:
    report_worksheet = report_sh.worksheet(report_id)

In [6]:
rdf['Date'] =  pd.DatetimeIndex(rdf.Date)
#rdf = rdf.drop(columns=["Notes", "Description", "Original Description", "Labels", "Notes"])

In [7]:
debits = rdf[rdf['Transaction Type'] == 'debit']

In [8]:
debits['Account Name'].unique()

array(['Credit (A)', 'Brokerage (V)', 'Checking (U)', 'IRA (V)',
       'HSA (F)', 'Credit (U4)', 'Checking (C)', 'Savings (E)',
       'Savings (C)', 'Savings (U)', 'Cash', 'Yahoo 401K (F)',
       'Brokerage (F)', 'Oath 401K (F) ', 'Checking (E)', 'Brokerage (E)',
       'Savings (A)', 'Credit (U3)', 'Credit (U2)', 'Corporate Card',
       'Yahoo 401K (V)', 'Loan (U2)', 'Credit (U1)', 'Loan (U1A)'],
      dtype=object)

In [9]:
accounts = [
    'Credit (A)',
    'Credit (U4)',
    'Credit (U3)',
    'Credit (U2)',
    'Credit (U1)',
    'Checking (C)',
    'Checking (U)',
    'Checking (E)',
    'Savings (C)',
    'Savings (U)',
    'Savings (E)',
    'Savings (A)',
    'Cash',
    'HSA (F)',
]

debits = debits[debits['Account Name'].isin(accounts)]

In [10]:
debits.Category.unique()


array(['Gift', 'Sales Tax', 'Shipping', 'Books', 'Credit Card Payment',
       'Home Supplies', 'Stationary', 'Clothing', 'Groceries',
       'Investments', 'Auto Accessories', 'Personal Care', 'Appliances',
       'Pharmacy', 'Tips', 'Restaurant', 'Accessories',
       'Renters Insurance', 'Internet', 'Electricity', 'Federal Tax',
       'TV', 'Investment Account', 'Kitchenware', 'Mobile Phone', 'Water',
       'Auto Insurance', 'Fuel', 'Transfer', 'Rent', 'Furniture',
       'Electronics', 'Home Improvement', 'Dentist', 'Doctor', 'Garden',
       'Subscriptions', 'Car Wash', 'Taxi', 'Rail Travel',
       'Entertainment', 'Home Decor', 'Toys', 'Bath Accessories',
       'License Renewal', 'Moving', 'Allowance', 'Donation', 'State Tax',
       'Movie', 'Software', 'Bank Fee', 'Music', 'Hotel', 'Bus Travel',
       'Transfer to NRE', 'Air Travel', 'Auto Service & Parts', 'ATM Fee',
       'Parking', 'Printing', 'Medical Lab', 'Travel', 'Museum', 'Legal',
       'Fitness', 'Service Fee',

In [11]:
ignored = [
    'Transfer',
    'Investments',
    'Buy',
    'Investment Account',
    'Buy',
    'Credit Card Payment',
    'Dividends & Cap Gains',
    'Transfer to NRE',
    'Allowance',
    'Withdrawal',
    'Option Exercise Cost',
    'Federal Tax',
    'State Tax'
]

In [12]:
df = debits[~debits.Category.isin(ignored)]
ydf = df.groupby([df.Date.dt.to_period('Y'), "Category"])['Amount'].sum().reset_index(name='Amount').sort_values(by=['Date', 'Amount'], ascending=False)

In [13]:
category_report = ydf.groupby(["Category"])['Amount'].sum().reset_index(name='Amount').sort_values(by=['Amount'], ascending=False)

pv_table = pd.pivot_table(ydf, index = 'Category', columns = 'Date', values = 'Amount', fill_value=0, dropna=False)


annual_monthly_report=category_report.merge(pv_table, on='Category')

annual_monthly_report.loc['Total'] = annual_monthly_report.sum(numeric_only=True)
annual_monthly_report = annual_monthly_report.fillna(value={"Category": "Total"}, limit=1).sort_values(by=['Amount'], ascending=False)

columns = annual_monthly_report.columns.values.tolist()
for idx, val in enumerate(columns):
    if isinstance(val, pd.Period):
        # pd.Period is not json serializable for writing to Google Sheets
        columns[idx] = val.strftime('%Y')
        
report_worksheet.update( [columns] + annual_monthly_report.values.tolist())
        

{'spreadsheetId': '1HqbpYwwo8vgz0k8NUPDX6V2WA_AOVOLutMrLWCofmUU',
 'updatedRange': 'all!A1:K69',
 'updatedRows': 69,
 'updatedColumns': 11,
 'updatedCells': 759}